In [1]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES=0,1,2,3
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [8]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [9]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [10]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(50000, 300)

In [11]:
!hostname
print(invalid)

cuda2.cims.nyu.edu
1493


In [12]:
c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

In [139]:
examples_to_take_train = 1000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [140]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [141]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
expression_contexts = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.int32)
expression_probabilities = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.float32)

In [142]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1
            
    words_to_consider_expression = set(X_train_comp[i] + nltkStopWords + punctuations)

    for j,word in enumerate(words_to_consider_expression):
        expression_contexts[i,:,look_up_word_reduced(word)] = 1
        
    for j,word in enumerate(words_to_consider_expression):
        expression_probabilities[i,:,look_up_word_reduced(word)] = len(np.where(expression_contexts[i][0] == 1)[0]) / float(wordToTake)
    expression_probabilities[i,:,np.where(expression_probabilities[i][0] == 0)[0]] = len(np.where(expression_contexts[i][0] == 0)[0]) / float(wordToTake)
    
        

(1000, 495)


In [143]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [144]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(50000, 300)
18192
1356
17654


In [145]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [146]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [147]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [160]:
max_document_len

495

## Batch Data Preparation

In [163]:
import math
def create_batch(inputs,batch_size,shuffle=False):
    num_batches = len(inputs[0]) // batch_size + 1
    outputs = []
    for index,inp in enumerate(inputs):
    
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_masks': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                    'suppression_answer':[],
                    'expression_contexts': [],
                    'expression_probabilities':[]}
    
        start = 0
        for i in range(num_batches):
            if i == num_batches - 1:
                end = None
            else:
                end = start+batch_size
            #maxD = max(inputs[1][start:end])
            maxD = max_document_len
            maxA = max(inputs[4][start:end])
            maxQ = max(inputs[7][start:end])
            if index == 0:
                outputs['document_tokens'] = inp[start:end,:maxD]
            elif index==1:
                outputs['document_lengths'] = inp[start:end]
            elif index == 2:
                outputs['answer_labels']=inp[start:end,:maxD]
            elif index==3:
                outputs['answer_masks']=inp[start:end,:maxA,:maxD]
            elif index==4:
                outputs['answer_lengths']=inp[start:end]
            elif index==5:
                output['question_input_tokens']=inp[start:end,:maxQ]
            elif index==6:
                output['question_output_tokens']=inp[start:end,:maxQ]
            elif index==7:
                output['question_lengths'] = inp[start:end]
            elif index==8:
                output['suppression_answer'] = inp[start:end]
            elif index==9:
                output['expression_contexts'] = inp[start:end,0:maxQ,:]
            elif index==10: 
                output['expression_probabilities'] = inp[start:end,0:maxQ,:]
            start = start + batch_size
        outputs.append(output)
    
    return outputs

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index==9 or index==10:
                output.append(inp[start:start+batch_size,0:maxQ,:])
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        #maxD = max(inputs[1][start:])
        maxD = max_document_len
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ])
        elif index==9 or index==10:
            output.append(inp[start:,0:maxQ,:]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [164]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer,expression_contexts,expression_probabilities]
                    ,batch_size)
number_of_batches = len(batch_input[0])
print("Number of batches = ", number_of_batches)

Number of batches =  32


## Model

In [165]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [166]:
use_cuda = torch.cuda.is_available()
use_attention = True

In [167]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        
        # TODO: Verify
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [168]:
class AnswerEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(AnswerEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(2, batch_size, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [169]:
class QuestionEncoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
    
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch.zeros(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [170]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result
    
'''
class FCLayer(nn.Module):
    def __init__(self,input_size, output_size):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        return self.fc(x)
'''

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x):
        output = self.fc(x)
        output = F.log_softmax(output, dim=-1)
        return output

In [171]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, max_document_len, dropout_p=0.1):
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_document_len = max_document_len

        self.attn_reduce = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.attn_combine = nn.Linear(self.max_document_len + self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        if use_cuda:
            self.attn_reduce = self.attn_reduce.cuda()
            self.attn_combine = self.attn_combine.cuda()
            self.dropout = self.dropout.cuda()
        

    def forward(self, input, hidden, encoder_outputs):

        hidden = hidden.squeeze(0)
        hidden = self.dropout(hidden)
        concat = torch.cat((input.unsqueeze(0), hidden.unsqueeze(0)), 1)
        concat_reduced = self.attn_reduce(concat)
        attn_weights = F.softmax(concat_reduced, dim=1)
        attn_applied = torch.mm(encoder_outputs, attn_weights.squeeze(0).unsqueeze(1))
        output = torch.cat((hidden.unsqueeze(0), attn_applied.squeeze(1).unsqueeze(0)), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        return output, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [172]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
#fcLayer = FCLayer(hidden_size, hidden_size)
answerEncoder = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)

answerEncoder.train()
questionEncoder.train()
questionDecoder.train()
questionGenerator.train()
attention.train()

train_param = []

for model in [answerEncoder, questionEncoder, questionDecoder, questionGenerator, attention]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 0.01)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

Number of trainable parameters =  24


In [173]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [ ]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None

if use_attention:
    Attention_Weights = None
    attn_output = None
    attention_hidden = attention.initHidden()

for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):

        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue


        #maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp).cuda()
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)


        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()


        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())


        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        question_encoder_hidden_batch = Variable(torch.zeros(1,current_batch_size,questionEncoder.hidden_size))
        if use_cuda:
            question_encoder_hidden_batch = question_encoder_hidden_batch.cuda()

        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        for i in range(current_batch_size):
            _ , question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
            question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden

        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        question_loss = 0
        for i in range(current_batch_size):
            question_decoder_hidden = question_encoder_hidden_batch[:,i:i+1,:].clone()
            embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()
            output_labels = Variable(torch.from_numpy(batch_input[5][batch_num][i]).long())
            if use_cuda:
                output_labels = output_labels.cuda()

            for quesL in range(batch_input[7][batch_num][i]):
                if use_attention:
                    attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                    decoder_output, attention_hidden = questionDecoder(
                        embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
                else:
                    decoder_output, question_decoder_hidden = questionDecoder(
                        embedded_inputs[quesL:quesL+1].unsqueeze(1),
                        question_decoder_hidden)

                final_output = questionGenerator(decoder_output)
                output_label = Variable(torch.zeros(1,reduced_glove.shape[0]))
                if use_cuda:
                    output_label = output_label.cuda()
                output_label[:,batch_input[5][batch_num][i][quesL]] = 1
                question_loss += criterion2(final_output.squeeze(0),
                                           output_labels[quesL:quesL+1])
                ##question_loss += criterion2(final_output.squeeze(0), output_label)


        net_loss = answer_loss + question_loss
        net_loss.backward(retain_graph=True)
        optimizer.step()


        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

Batch: 0 	 Epoch : 1	Net Loss: 4331.4443 	Answer Loss: 0.1946 	Question Loss: 4331.2500
Batch: 1 	 Epoch : 1	Net Loss: 4048.8394 	Answer Loss: 0.2927 	Question Loss: 4048.5466
Batch: 2 	 Epoch : 1	Net Loss: 3249.6189 	Answer Loss: 0.2281 	Question Loss: 3249.3909
Batch: 3 	 Epoch : 1	Net Loss: 6511.0171 	Answer Loss: 0.2667 	Question Loss: 6510.7505
Batch: 4 	 Epoch : 1	Net Loss: 6210.5737 	Answer Loss: 0.3230 	Question Loss: 6210.2510
Batch: 5 	 Epoch : 1	Net Loss: 3065.8022 	Answer Loss: 0.2933 	Question Loss: 3065.5090
Batch: 6 	 Epoch : 1	Net Loss: 5012.9780 	Answer Loss: 0.3228 	Question Loss: 5012.6553
Batch: 7 	 Epoch : 1	Net Loss: 3935.1260 	Answer Loss: 0.2287 	Question Loss: 3934.8972
Batch: 8 	 Epoch : 1	Net Loss: 2470.7983 	Answer Loss: 0.1509 	Question Loss: 2470.6475
Batch: 9 	 Epoch : 1	Net Loss: 2613.9558 	Answer Loss: 0.2740 	Question Loss: 2613.6819
Batch: 10 	 Epoch : 1	Net Loss: 2279.8730 	Answer Loss: 0.3314 	Question Loss: 2279.5417
Batch: 11 	 Epoch : 1	Net Loss:

Batch: 0 	 Epoch : 4	Net Loss: 163.9716 	Answer Loss: 0.1310 	Question Loss: 163.8406
Batch: 1 	 Epoch : 4	Net Loss: 324.5998 	Answer Loss: 0.2668 	Question Loss: 324.3329
Batch: 2 	 Epoch : 4	Net Loss: 370.7748 	Answer Loss: 0.2145 	Question Loss: 370.5604
Batch: 3 	 Epoch : 4	Net Loss: 323.2723 	Answer Loss: 0.2379 	Question Loss: 323.0344
Batch: 4 	 Epoch : 4	Net Loss: 272.5392 	Answer Loss: 0.2737 	Question Loss: 272.2656
Batch: 5 	 Epoch : 4	Net Loss: 215.4164 	Answer Loss: 0.2423 	Question Loss: 215.1741
Batch: 6 	 Epoch : 4	Net Loss: 186.8780 	Answer Loss: 0.2999 	Question Loss: 186.5781
Batch: 7 	 Epoch : 4	Net Loss: 243.7474 	Answer Loss: 0.2251 	Question Loss: 243.5223
Batch: 8 	 Epoch : 4	Net Loss: 150.5847 	Answer Loss: 0.1463 	Question Loss: 150.4384
Batch: 9 	 Epoch : 4	Net Loss: 116.5801 	Answer Loss: 0.2618 	Question Loss: 116.3183
Batch: 10 	 Epoch : 4	Net Loss: 107.7460 	Answer Loss: 0.3266 	Question Loss: 107.4195
Batch: 11 	 Epoch : 4	Net Loss: 125.4510 	Answer Loss

In [177]:
answer_tags.shape, t_document_mask.shape

(torch.Size([32, 495, 1]), torch.Size([32, 224]))

In [ ]:
checkpoint = torch.load("/data/ra2630/qgen_base_40k.pt")

In [ ]:
answerEncoder2 = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder2 = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator2 = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
optimizer2 = torch.optim.Adam(train_param, 0.01)

#answerEncoder2.load_state_dict(checkpoint["answerEncoder"])
#questionEncoder2.load_state_dict(checkpoint["questionEncoder"])
#questionDecoder2.load_state_dict(checkpoint["questionDecoder"])
#questionGenerator2.load_state_dict(checkpoint["questionGenerator"])
#optimizer2.load_state_dict(checkpoint["optimizer"])

answerEncoder2 = answerEncoder
questionEncoder2 = questionEncoder
questionDecoder2 = questionDecoder
questionGenerator2 = questionGenerator
optimizer2 = optimizer

In [130]:
encodedAnswerOutputs = answerEncoder(embedded_inp[:,:,:], answerEncoder.initHidden())

In [138]:
np.where(encodedAnswerOutputs[0][0].data > 0.5)

(array([ 6, 19, 20, 21, 22, 23, 24, 41]), array([0, 0, 0, 0, 0, 0, 0, 0]))

In [137]:
np.where(batch_input[2][0][0] == 1)

(array([ 5,  6, 19, 20, 21, 22, 23, 24, 34, 35]),)

In [ ]:
def printQuestion(batch_num, example_num):
    for i in batch_input[6][batch_num][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")
        
def printAnswer(batch_num, example_num):
    for i in batch_input[3][batch_num][example_num][0 : batch_input[4][batch_num][example_num]]:
        for j in (np.where(i==1)[0]):
            print(look_up_token_reduced(batch_input[0][batch_num][example_num][j]))
    print("")
        
def printContext(batch_num, example_num):
    for i in batch_input[0][batch_num][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")

In [ ]:
batch_num = 0
example_num=1
print("Context : ")
printContext(batch_num, example_num)
print("Question : ")
printQuestion(batch_num, example_num)
print("Answer : ")
printAnswer(batch_num, example_num)

In [ ]:
question_token = START_TOKEN
question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
qLen = 0
while qLen <= 20:
    decoder_output, question_decoder_hidden = questionDecoder(
        question_token_embedding.unsqueeze(1),
        question_decoder_hidden)
    final_output = questionGenerator(decoder_output)
    question_token = np.argmax(final_output.data)
    print(question_token)
    question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
    qLen=qLen + 1
    print(look_up_token_reduced(question_token))

In [ ]:
def inference(batch_number, example_number, context, answer = None):

    inp = Variable(torch.from_numpy(batch_input[0][batch_number][example_number]).long())
    embedded_inp = embedder(inp).cuda()
    answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
    answer_tags, answer_outputs, _ = answerEncoder(embedded_inp, answer_encoder_hidden)


    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda()


    t_document_mask = Variable(torch.from_numpy(mask)).float()
    if use_cuda:
        t_document_mask = t_document_mask.cuda()
    outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


    answer_loss = criterion1(outputs, labels.float())


    t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
    if use_cuda:
        t_answer_mask = t_answer_mask.cuda()

    question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
    question_encoder_hidden_batch = Variable(torch.zeros(1,current_batch_size,questionEncoder.hidden_size))
    if use_cuda:
        question_encoder_hidden_batch = question_encoder_hidden_batch.cuda()

    question_encoder_hidden = repackage_hidden(question_encoder_hidden)
    for i in range(current_batch_size):
        _ , question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
        question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden

    if type(question_decoder_hidden) == Variable:
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
    question_loss = 0
    for i in range(current_batch_size):
        question_decoder_hidden = question_encoder_hidden_batch[:,i:i+1,:].clone()
        embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()
        output_labels = Variable(torch.from_numpy(batch_input[5][batch_num][i]).long())
        if use_cuda:
            output_labels = output_labels.cuda()

        for quesL in range(batch_input[7][batch_num][i]):
            decoder_output, question_decoder_hidden = questionDecoder(
                embedded_inputs[quesL:quesL+1].unsqueeze(1),
                question_decoder_hidden)

            final_output = questionGenerator(decoder_output)
            output_label = Variable(torch.zeros(1,reduced_glove.shape[0]))
            if use_cuda:
                output_label = output_label.cuda()
            output_label[:,batch_input[5][batch_num][i][quesL]] = 1
            question_loss += criterion2(final_output.squeeze(0),
                                       output_labels[quesL:quesL+1])
    

In [ ]:

##########ONly Batch

verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None
for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):
        
        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue
        
        
        maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

            
        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        ################################### Answer Encoder + Tagging    

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        if use_cuda:
            embedded_inp = embedder(inp).cuda()

            
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)
        

        
        
        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()

        #answer_tags.requires_grad=False
        #answer_outputs.requires_grad=False
        
        
        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        #t_document_mask.requires_grade = False
        
        outputs = answer_tags.squeeze(-1) * t_document_mask
        #outputs.requires_grad = False
        
        
        answer_loss = criterion1(outputs, labels.float())
        ################################### Q Encoder    
        
        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        #t_answer_mask.requires_gradui = False
        
        # masking the non-answer embeddings
        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        #question_encoder_input.requires_grad = False
        question_encoder_output = Variable(torch.zeros(1,batch_size,questionEncoder.hidden_size))
        #question_encoder_output.requires_grad = False
        
        
        if use_cuda:
            question_encoder_hidden = question_encoder_hidden_batch.cuda()
            
        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input, question_encoder_hidden)
        
        #question_encoder_output.requires_grad = False

        
#         question_encoder_output = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
#         question_encoder_hidden = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
        
#         for i in range(current_batch_size):
#             question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
#             question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden    

        ################################### Q Decoder    
        
        question_loss = 0
        
        question_decoder_hidden = question_encoder_hidden
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        
        embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num]).long())
        #embedded_inputs.requires_grad = False
        
        output_labels = Variable(torch.from_numpy(batch_input[6][batch_num]).long())
        
        decoder_output = Variable(torch.zeros(1,batch_size,questionDecoder.hidden_size))
        
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()
            
        
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        for quesL in range(max(batch_input[7][batch_num])):
            decoder_output, question_decoder_hidden = questionDecoder(
                embedded_inputs[:,quesL:quesL+1,:],
                question_decoder_hidden)

            final_output = questionGenerator(decoder_output)

            output_label = Variable(torch.zeros(32,2000))
            if use_cuda:
                output_label = output_label.cuda()
            for b in range(len(batch_input[5][0])):
                output_label[b,batch_input[5][batch_num][b][quesL]] = 1
            #question_loss += criterion2(final_output.squeeze(1), 
            #                           output_labels[:,quesL:quesL+1])
            question_loss += criterion2(final_output.squeeze(0), output_label)
        
    
        
    
        net_loss = answer_loss + question_loss
        net_loss.backward()
    
        optimizer.step()
        
        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f' 
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))
            
    torch.cuda.empty_cache()
          
        
    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))